In [8]:
# Remember: comments in python are denoted by the pou_oldd sign
import numpy as np                    #here we load numpy
from matplotlib import pyplot      #here we load matplotlib
import time, sys                   #and load some utilities

import math

In [9]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator

%matplotlib inline

nx = 41
ny = 41
nt = 100
ni = 50

rho = 1
nu = .1
dt = .001

dx = 2 / (nx - 1)
dy = 1 / (ny - 1)

p = np.empty((ny, nx))
p0 = np.empty((ny, nx))

u = np.empty((ny, nx))
v = np.empty((ny, nx))

u0 = np.empty((ny, nx))
v0 = np.empty((ny, nx))

b = np.empty((ny, nx))

# initial condition
for i in range(ny):
    for j in range(nx):
        p[i, j] = 0
        b[i, j] = 0
        u[i, j] = 0
        v[i, j] = 0

u[-1, :] = 1 # u = 1 at y = 2

# u = v = 0 at x = 0, 1, y = 0, v = 0 at y = 2
u[0, :] = u[:, 0] = u[:, -1] = v[0, :] = v[:, 0] = v[:, -1] = v[-1, :] = 0 

p[-1, :] = 0 # p = 0 at y = 2
p[0, :] = p[1, :] # dp / dy = 0 at y = 0
p[:, -1] = p[:, -2] # dp / dx = 0 at x = 2
p[:, 0] = p[:, 1] # dp / dx = 0 at x = 0

# grid
x = np.empty(nx)
y = np.empty(ny)

for i in range(nx):
    x[i] = i * dx

for j in range(ny):
    y[j] = j * dy

In [10]:
bCoeff = ((rho * dx**2 * dy**2) / (2 * (dx**2 + dy**2)))
for t in range(nt):
    u0 = u.copy()
    v0 = v.copy()
    
    # Solve Poisson pressure equation

    # Calculate b
    for i in range(1, ny-1):
        for j in range(1, nx-1):
            b[i, j] = bCoeff * ((1. / dt) * (((u[i+1, j] - u[i-1, j]) / (2*dx)) + ((v[i, j+1] - v[i, j-1]) / (2*dy))))
            b[i, j] -= bCoeff * (((u[i+1, j] - u[i-1, j]) / (2.*dx)) * ((u[i+1, j] - u[i-1, j]) / (2.*dx)))
            b[i, j] -= bCoeff * 2 * (((v[i+1, j] - v[i-1, j]) / (2*dx)) * ((u[i, j+1] - u[i, j-1]) / (2*dy)))
            b[i, j] -= bCoeff * (((v[i, j+1] - v[i, j-1]) / (2*dy)) * ((v[i, j+1] - v[i, j-1]) / (2*dy)))
    
    # Solve for p
    for n in range(ni):
        p0 = p.copy()
        for i in range(1, ny-1):
            for j in range(1, nx-1):
                p[i, j] = (((p0[i+1, j] + p0[i-1, j]) * dy**2) + ((p0[i, j+1] + p0[i, j-1]) * dx**2)) / (2 * (dx**2 + dy**2)) - b[i, j]

        p[:, -1] = p[:, -2] # dp/dx = 0 at x = 2
        p[0, :] = p[1, :]   # dp/dy = 0 at y = 0
        p[:, 0] = p[:, 1]   # dp/dx = 0 at x = 0
        p[-1, :] = 0        # p = 0 at y = 2

    # velocity calculation
    for i in range(1, ny-1):
        for j in range(1, nx-1):
            u[i, j] = u0[i, j] - ((dt / rho * dx) * (p[i+1, j] - p[i-1, j])) - (u0[i, j] * (dt / dx) * (u0[i, j] - u0[i-1, j]))
            u[i, j] -= v0[i, j] * (dt / dy) * (u0[i, j] - u0[i, j-1]) + ((nu * dt) / dx**2) * (u0[i+1, j] - 2 * u0[i, j] + u0[i-1, j]) + ((nu * dt) / dy**2) * (u0[i, j+1] - 2 * u0[i, j] + u0[i, j-1])

            v[i, j] = v0[i, j] - ((dt / rho * dx) * (p[i, j+1] - p[i, j-1])) - (u0[i, j] * (dt / dx) * (v0[i, j] - v0[i-1, j]))
            v[i, j] -= v0[i, j] * (dt / dy) * (v0[i, j] - v0[i, j-1]) + ((nu * dt) / dx**2) * (v0[i+1, j] - 2 * v0[i, j] + v0[i-1, j]) + ((nu * dt) / dy**2) * (v0[i, j+1] - 2 * v0[i, j] + v0[i, j-1])

    # Set boundary conditions
    u[-1, :] = 1 # u = 1 at y = 2

    # u = v = 0 at x = 0, 1, y = 0, v = 0 at y = 2
    u[0, :] = u[:, 0] = u[:, -1] = v[0, :] = v[:, 0] = v[:, -1] = v[-1, :] = 0 



C:\Users\suraj\AppData\Local\Temp\ipykernel_2704\4231361978.py:12: RuntimeWarning: overflow encountered in scalar multiply
  b[i, j] -= bCoeff * (((u[i+1, j] - u[i-1, j]) / (2.*dx)) * ((u[i+1, j] - u[i-1, j]) / (2.*dx)))
C:\Users\suraj\AppData\Local\Temp\ipykernel_2704\4231361978.py:13: RuntimeWarning: overflow encountered in scalar multiply
  b[i, j] -= bCoeff * 2 * (((v[i+1, j] - v[i-1, j]) / (2*dx)) * ((u[i, j+1] - u[i, j-1]) / (2*dy)))
C:\Users\suraj\AppData\Local\Temp\ipykernel_2704\4231361978.py:13: RuntimeWarning: invalid value encountered in scalar subtract
  b[i, j] -= bCoeff * 2 * (((v[i+1, j] - v[i-1, j]) / (2*dx)) * ((u[i, j+1] - u[i, j-1]) / (2*dy)))
C:\Users\suraj\AppData\Local\Temp\ipykernel_2704\4231361978.py:14: RuntimeWarning: overflow encountered in scalar multiply
  b[i, j] -= bCoeff * (((v[i, j+1] - v[i, j-1]) / (2*dy)) * ((v[i, j+1] - v[i, j-1]) / (2*dy)))
C:\Users\suraj\AppData\Local\Temp\ipykernel_2704\4231361978.py:31: RuntimeWarning: overflow encountered in sc

In [12]:
X, Y = numpy.meshgrid(x, y)

fig = pyplot.figure(figsize=(11,7), dpi=100)
# plotting the pressure field as a contour
pyplot.contourf(X, Y, p, alpha=0.5, cmap=cm.viridis)  
pyplot.colorbar()
# plotting the pressure field outlines
pyplot.contour(X, Y, p, cmap=cm.viridis)  
# plotting velocity field
pyplot.quiver(X[::2, ::2], Y[::2, ::2], u[::2, ::2], v[::2, ::2]) 
pyplot.xlabel('X')
pyplot.ylabel('Y');

NameError: name 'numpy' is not defined